# GASKAP - Absorption HI: LMC foreground

# Libraries

In [1]:
import os
import sys
import glob


import numpy as np
import pandas as pd
import copy as cp
import healpy as hp

import scipy
import scipy.signal

from astropy import units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.io.votable import from_table, parse_single_table, parse
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle


from spectral_cube import SpectralCube

import pickle
import shutil

import matplotlib.pyplot as plt
import matplotlib        as mpl
from matplotlib.patches import Ellipse
from matplotlib.ticker import AutoMinorLocator

import matplotlib.pyplot as plt
from matplotlib import patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec

from pathlib import Path
import more_itertools as mit

import seaborn as sns

In [ ]:
# import libs.gfitflex as fitlib
script_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.abspath(Path(script_dir) / os.pardir)
print('script_dir: ', script_dir)
print('parent_dir: ', parent_dir)
gaussfit_dir = Path(parent_dir) / 'gaussfit'
print('gaussfit_dir: ', gaussfit_dir)
sys.path.append(parent_dir)
module_dir = Path(parent_dir) / 'gaussfit' / 'modules'
sys.path.append(module_dir)

print('script_dir: ', script_dir)
print('parent_dir: ', parent_dir)
print('gaussfit_dir: ', gaussfit_dir)

script_dir

In [3]:
import libs.functions as fcn
import gaussfit.modules.fit as find_gcomp
import libs.fitlibs as fitlibs
import libs.functions as fcn
import config.config as cf

In [4]:
# Set the font family for math symbols
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'

plt.rc('text', usetex=False)
plt.rc('font', family='serif')

# Display all colums
pd.set_option('display.max_columns', None)

# Paths

In [ ]:
base = Path.home()
askap = base / 'ANU' / 'ASKAP'
data_path = askap / 'data'
LMC_data_path = data_path / 'LMC'
LMC_GASKAP_data = LMC_data_path / 'lmc_gaskap'

cont_data_path = data_path / 'cont'
LMC_bridge_data = LMC_data_path / 'bridge'
GASS_em_spectra_path = LMC_data_path / 'neighbour_emission_spectra'
GASS_data_path = LMC_data_path / 'GASS'
HI_data_path = data_path / 'GASS'

SMC = askap / 'SMC'
SMC_data_path = data_path / 'SMC'
SMC_fg_data_path = SMC_data_path / 'fg'
SMC_fg_abs_data_path = SMC_fg_data_path / 'spectra'

hires_data = LMC_data_path / 'hires'
hires_results = hires_data / 'results'
hires_em_data = hires_data / 'cube'
hires_em_subcube_data = hires_em_data / 'subcubes'
hires_abs_data = hires_data / 'abs'
em_spectra_path = hires_data / 'neighbour_emission_spectra'
hires_em_profile_path = hires_data / 'em_profiles'
abs_dr3_data_path = hires_data / 'absdr3'

mwmc_data_path = data_path / 'mwmc'
mwmc_gaskap_path = mwmc_data_path / 'lmc_gaskap'

bridge_hires_abs_data = hires_abs_data / 'bridge'
LMC_hires_abs_data = hires_abs_data / 'lmc'

abs_path = mwmc_data_path / 'absorption'

BIGHICAT_data_path = data_path / 'BIGHICAT'


dust_data_path = data_path / 'dust'

# Absorption catalog DR3
print(abs_dr3_data_path)

# Fcn

In [47]:
# linear model
def linear_fcn(p, x):
    m, b = p
    return m*x + b

In [48]:
# Define a function to convert decimal hours to hh:mm:ss format
def convert_ra(ra):
    hours = int(ra)
    minutes = int((ra - hours) * 60)
    seconds = round(((ra - hours) * 60 - minutes) * 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


In [49]:
# Define a function to convert decimal degrees to dd:mm:ss format
def convert_dec(dec):
    degrees = int(dec)
    minutes = int((abs(dec) - abs(degrees)) * 60)
    seconds = round(((abs(dec) - abs(degrees)) * 60 - minutes) * 60)
    sign = '-' if dec < 0 else '+'
    return f"{sign}{abs(degrees):02d}:{minutes:02d}:{seconds:02d}"

In [50]:
def get_sbid(x):
    ret = int(x.split('_')[0].replace('SB', ''))
    return ret

In [51]:
def binned_avg(x: np.ndarray, y: np.ndarray, bin_edges: np.ndarray, n_resamples = 1000):
    from scipy import stats

    # Calculate the bin average using scipy
    bin_avg, _, _ = stats.binned_statistic(x, y, statistic='mean', bins=bin_edges)

    # Container for bin averages
    bin_avg_arr = []

    # Perform bootstrap resampling and calculate bin averages
    for _ in range(n_resamples):
        resample_indices = np.random.choice(len(y), size=len(y), replace=True)
        resample_y= y[resample_indices]
        bin_avg_resample, _, _ = stats.binned_statistic(x, resample_y, statistic='mean', bins=bin_edges)
        bin_avg_arr.append(bin_avg_resample)

    # Compute the upper and lower percentiles of the bin averages
    bin_avg_arr = np.array(bin_avg_arr)
    bin_mean_arr = np.mean(bin_avg_arr, axis=0)
    bin_avg_upper = np.percentile(bin_avg_arr, 97.5, axis=0) - bin_mean_arr + bin_avg
    bin_avg_lower = np.percentile(bin_avg_arr, 2.5, axis=0) - bin_mean_arr + bin_avg

    return bin_avg, bin_avg_upper, bin_avg_lower

In [52]:
class Source:
    # A class for a source
        
    # Initialize the object
    def __init__(self,
                 src,
                 df = None,
                 field_id = 0,
                 id = 0
                ):        
        
        # source
        self.src      = src
        self.field_id = field_id
        self.field    = 'sb' + str(field_id)
        self.id       = id

        # Default info
        info = {}
        info['flux_int']      = 0.
        info['semi_maj_axis'] = 0.
        info['semi_min_axis'] = 0.
        info['id']            = id
        info['glon']          = 0.
        info['glat']          = 0.
        info['ra']            = 0.
        info['dec']           = 0.
        info['tsky']          = 2.73 # K
        info['rating']        = None
        info['has_mw_abs']    = 0
        info['sbid']          = field_id

        self.info = info

        if df is not None:
            self.df = df
            info = self.get_src_from_name()

        self.info = info
        self.id = 0 # info['id']
        self.field_id = info['sbid']
        self.field = 'sb' + str(self.field_id)
        self.l = info['glon']
        self.b = info['glat']
        self.ra = info['ra']
        self.dec = info['dec']
        self.tsky = info['tsky']
        self.rating = info['rating']
        self.has_mw_abs = 1 # info['has_mw_abs']


        # Absorption dir
        self.abs_filename = hires_abs_data / self.field / 'spectra' / (self.src + '_spec.vot')

        # Emission data and info of the source exist?
        self.csv_file = em_spectra_path / self.field / (self.src + '.csv')
        self.pkl_file = em_spectra_path / self.field / (self.src + '.pkl')
        self.em_file_conditions = (os.path.exists(self.csv_file) & os.path.exists(self.pkl_file) & os.path.exists(self.abs_filename))



    def __repr__(self):
        out = self.__class__.__name__ + '('
        out += 'Source=%s, field=%s, l,b=[%s,%s], radec=[%s,%s])\n' % \
               (self.src, self.field, self.l, self.b, self.ra, self.dec)
        return out

    def __str__(self):
        out = self.__class__.__name__ + '('
        out += 'Source=%s, field=%s, l,b=[%s,%s], radec=[%s,%s])\n' % \
               (self.src, self.field, self.l, self.b, self.ra, self.dec)
        return out
    

    
    
    
    def get_src_from_name(self):
        src_info = self.df.loc[self.df["comp_name"] == self.src]
        nfound = len(src_info['comp_name'].unique())
        
        if nfound == 0:
            print("Error: No source found!")
            return None
        
        return src_info.iloc[0]
    
    
    
    
    
    def get_src_infor(self):
        src_info = self.df.loc[(self.df["comp_name"] == self.src) & (self.df["sbid"] == self.field_id)]
        nfound = len(src_info['comp_name'].unique())
        
        if nfound == 0:
            print("Error: No source found!")
            return None

        if nfound == 1:
            return src_info.iloc[0]
        else:
            print("Error: source is not unique!")
            return None

In [ ]:
def read_spectra(src, abs_file, vel_idx):
    """
    Read spectrum from file.
    
    Parameters:
    filepath (string): Path to file

    Returns:
    A Profile object

    """

    # GASKAP ABSORPTION: VLSR of GASKAP absorption does not linearly separates. Need to exclude the last VLSR value.
    data = fitlibs.votable_to_pandas(abs_file)
    vlsr_gaskap = data['velocity'].values[:-1] / 1000.
    key = 'optical_depth' if 'optical_depth' in data.columns else 'opacity'
    sigkey = 'sigma_od' if 'sigma_od' in data.columns else 'sigma_opacity'
    emt    = data[key].values[:-1]
    sigemt = data[sigkey].values[:-1]
    vlsr_gaskap = vlsr_gaskap[vel_idx]

    # Absorption profiles
    emt    = emt[vel_idx]
    sigemt = sigemt[vel_idx]

    return vlsr_gaskap, emt, sigemt

# Read absorption catalogue DR3beta

In [71]:
# All sightlines
MC_fg_cat_file = abs_dr3_data_path / 'gaskap_abs_spectra_dr3b.csv'
df = pd.read_csv(MC_fg_cat_file)

nrows, ncols = df.shape
print('Number of rows: %i, columns %i. ' %(nrows, ncols))
df.head(5)

Number of rows: 2714, columns 33. 


,id,comp_name,component_id,ra,dec,glon,glat,rating,flux_peak,flux_int,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,has_mw_abs,has_other_abs,semi_maj_axis,semi_min_axis,pa,n_h,noise_flag,continuum_slope,field,all_sbids,sbid,mw_detection,saturated
0,0,J005611-710706,SB30665_component_1a,14.047482,-71.118446,302.378462,-46.004698,A,519.130005,442.507996,0.003324,0.004476,0.120743,5.142489,1.019657,144.0,0.898914,-0.788959,0.106568,0.005696,True,True,4.80,4.215,33.869999,1.236612e+21,False,0.000923,M344-11,30665,30665,True,False
1,8,J011049-731427,SB30665_component_3a,17.707468,-73.240921,300.996592,-43.814386,A,457.480988,596.744995,0.001633,0.005186,0.472219,23.407247,1.019347,119.0,0.547128,158.554968,0.603072,0.014683,True,True,5.79,5.350,54.860001,4.874624e+21,False,-0.000770,M344-11,30665,30665,True,False
2,9,J011056-731406,SB30665_component_3b,17.735945,-73.235151,300.984436,-43.819266,A,115.037003,208.171005,0.000281,0.022776,0.478756,5.717874,1.071266,93.0,0.592510,159.534285,0.523387,0.058836,False,True,7.48,5.745,39.880001,4.874624e+21,False,-0.000363,M344-11,30665,30665,False,False
3,10,J004048-714600,SB30665_component_4a,10.200474,-71.766489,304.115161,-45.337319,A,428.450012,463.325989,0.002056,0.008573,0.113398,3.477934,1.025324,76.0,0.911926,0.889198,0.092196,0.011053,True,False,5.23,4.905,175.600006,4.498524e+20,False,0.000679,M344-11,30665,30665,True,False
4,11,J005732-741242,SB30665_component_5a,14.385728,-74.211897,302.365011,-42.909634,A,428.031006,461.653015,0.001720,0.008202,0.514969,23.851180,1.020722,65.0,0.505753,139.606872,0.681707,0.017343,True,True,5.23,4.895,175.509995,5.650006e+20,False,0.000971,M344-11,30665,30665,True,False


In [72]:
print(df.info())
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2714 entries, 0 to 2713
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2714 non-null   int64  
 1   comp_name        2714 non-null   object 
 2   component_id     2714 non-null   object 
 3   ra               2714 non-null   float64
 4   dec              2714 non-null   float64
 5   glon             2714 non-null   float64
 6   glat             2714 non-null   float64
 7   rating           2714 non-null   object 
 8   flux_peak        2714 non-null   float64
 9   flux_int         2714 non-null   float64
 10  mean_cont        2714 non-null   float64
 11  sd_cont          2714 non-null   float64
 12  opacity_range    2714 non-null   float64
 13  max_s_max_n      2714 non-null   float64
 14  max_noise        2714 non-null   float64
 15  num_chan_noise   2714 non-null   float64
 16  min_opacity      2714 non-null   float64
 17  vel_min_opacit

id                 0
comp_name          0
component_id       0
ra                 0
dec                0
glon               0
glat               0
rating             0
flux_peak          0
flux_int           0
mean_cont          0
sd_cont            0
opacity_range      0
max_s_max_n        0
max_noise          0
num_chan_noise     0
min_opacity        0
vel_min_opacity    0
peak_tau           0
e_peak_tau         0
has_mw_abs         0
has_other_abs      0
semi_maj_axis      0
semi_min_axis      0
pa                 0
n_h                0
noise_flag         0
continuum_slope    0
field              0
all_sbids          0
sbid               0
mw_detection       0
saturated          0
dtype: int64

# Summary: Read table of integrated properties

In [93]:
csv_filename   = hires_results / 'hires_result_table_dr3.csv' # DR3
gfit_result_df = pd.read_csv(csv_filename)

ndet_total, ncols = gfit_result_df.shape

print('Number of sightlines with detections: ', ndet_total, '| ncols: ', ncols)
gfit_result_df.head(5)

Number of sightlines with detections:  462 | ncols:  42


,comp_name,sbid,uniq_name,ra,dec,glon,glat,in_bridge,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,semi_maj_axis,semi_min_axis,pa,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,n_h,ew,sigew,ewfit,sigewfit,nhicnm,signhicnm,nhiwnm,signhiwnm,fcnm,sigfcnm,Rfactor,sigRfactor,nhi,signhi,nhi_thin,signhi_thin,ncnm,tsky
0,J005611-710706,30665,J005611-710706-30665,14.047482,-71.118446,302.378462,-46.004698,0,A,519.130005,0.003324,0.004476,0.120743,5.142489,1.019657,144.0,4.800,4.215,33.869999,0.898914,-0.788959,0.106568,0.005696,1.236612e+21,0.214960,0.227081,0.227081,0.006692,0.258148,0.017671,1.974700,0.100309,0.114780,0.009487,0.991403,0.050255,2.232849,0.101853,2.251126,0.048945,1,3.543176
1,J011049-731427,30665,J011049-731427-30665,17.707468,-73.240921,300.996592,-43.814386,0,A,457.480988,0.001633,0.005186,0.472219,23.407247,1.019347,119.0,5.790,5.350,54.860001,0.547128,158.554968,0.603072,0.014683,4.874624e+21,0.086563,0.189310,0.189310,0.015265,0.673371,0.081768,2.090209,0.275080,0.244077,0.038535,1.033431,0.116433,2.763580,0.286976,2.673219,0.120832,2,3.573591
2,J004048-714600,30665,J004048-714600-30665,10.200474,-71.766489,304.115161,-45.337319,0,A,428.450012,0.002056,0.008573,0.113398,3.477934,1.025324,76.0,5.230,4.905,175.600006,0.911926,0.889198,0.092196,0.011053,4.498524e+20,0.378402,0.376982,0.376982,0.015596,1.038635,0.153567,1.943385,0.808341,0.346858,0.114980,1.121981,0.333828,2.982020,0.822799,2.679519,0.113766,1,3.564467
3,J005732-741242,30665,J005732-741242-30665,14.385728,-74.211897,302.365011,-42.909634,0,A,428.031006,0.001720,0.008202,0.514969,23.851180,1.020722,65.0,5.230,4.895,175.509995,0.505753,139.606872,0.681707,0.017343,5.650006e+20,0.297762,0.222502,0.222502,0.015980,0.359050,0.038501,2.377233,0.134253,0.130709,0.015873,1.013608,0.058216,2.736283,0.139665,2.701180,0.069134,1,3.485386
4,J004741-753010,30665,J004741-753010-30665,11.920932,-75.502942,303.246204,-41.623018,0,A,305.752991,0.001378,0.012283,0.283696,8.138546,1.031044,90.0,5.265,4.940,171.570007,0.747348,-1.632904,0.291224,0.019113,2.560501e+20,0.785208,0.561212,0.561212,0.013125,0.868798,0.054916,2.789718,0.574751,0.236581,0.042168,0.996616,0.161232,3.658516,0.577369,3.670527,0.119231,1,3.473220


In [96]:
gfit_result_df.columns

Index(['comp_name', 'sbid', 'uniq_name', 'ra', 'dec', 'glon', 'glat',
       'in_bridge', 'rating', 'flux_peak', 'mean_cont', 'sd_cont',
       'opacity_range', 'max_s_max_n', 'max_noise', 'num_chan_noise',
       'semi_maj_axis', 'semi_min_axis', 'pa', 'min_opacity',
       'vel_min_opacity', 'peak_tau', 'e_peak_tau', 'n_h', 'ew', 'sigew',
       'ewfit', 'sigewfit', 'nhicnm', 'signhicnm', 'nhiwnm', 'signhiwnm',
       'fcnm', 'sigfcnm', 'Rfactor', 'sigRfactor', 'nhi', 'signhi', 'nhi_thin',
       'signhi_thin', 'ncnm', 'tsky'],
      dtype='object')

In [111]:
# conver R.A to hours
gfit_result_df['ra_hr'] = gfit_result_df.ra/15.

In [ ]:
# Foreground velocity range: [-80, +80] km/s
# Velocity indices: [308, 472], 164 channels
vel_idx = [x for x in range(308, 472, 1)]
len(vel_idx)

In [112]:
shuffled_df = gfit_result_df.sample(frac=1).reset_index(drop=True)
shuffled_df.sort_values(by='ra', ascending=True, inplace=True)

srcnames = shuffled_df.comp_name.values
srcra = shuffled_df['ra_hr'].apply(convert_ra).values
srcdec = shuffled_df['dec'].apply(convert_dec).values
srcflux = shuffled_df.flux_int.values
srcfields = shuffled_df.sbid.values
srcbmaj = shuffled_df.semi_maj_axis.values
srcbmin = shuffled_df.semi_min_axis.values
srctsky = shuffled_df.tsky.values

srcgl = shuffled_df.glon.values
srcgb = shuffled_df.glat.values

dv = 0.9771557379632014 # km/s
tau_noise_per_channel = []
for src in srcnames:
    # step 1: information of a source
    source = Source(src, gfit_result_df)
    vlsr, emt, sigemt = read_spectra(src, source.abs_filename, vel_idx)
    idx = [x for x in range(0, 10, 1)] + [x for x in range(len(vlsr)-10, len(vlsr), 1)]
    sigtau = sigemt[idx] / emt[idx]
    tau_noise_per_channel.append(np.median(sigtau/dv))

print(np.min(tau_noise_per_channel), np.median(tau_noise_per_channel), np.mean(tau_noise_per_channel),  np.max(tau_noise_per_channel))

0.0019132457033010021 0.04426785386396564 0.05378718347302121 0.1978066414987632


# Table 1 in the paper

In [361]:
# Table 1 in the paper
for i, (srci, fieldi, rai, deci, li, bi, fluxi, noisei, tskyi) in enumerate(zip(srcnames, srcfields, srcra, srcdec, srcgl, srcgb, srcflux, tau_noise_per_channel, srctsky)):
    print(f'{srci} & {fieldi} & {rai} & {deci} & {li:0.2f} & {bi:0.2f} & {fluxi:0.1f} & {noisei:0.3f} & {tskyi:0.2f} \\\\')
    if i > 50:
        break

J001424-733911 & 30665 & 00:14:24 & -73:39:11 & 306.49 & -43.22 & 179.5 & 0.015 & 3.56 \\
J002144-741500 & 30665 & 00:21:44 & -74:14:59 & 305.67 & -42.72 & 132.4 & 0.028 & 3.56 \\
J002223-742825 & 30665 & 00:22:23 & -74:28:26 & 305.56 & -42.51 & 206.7 & 0.019 & 3.56 \\
J002248-734007 & 30665 & 00:22:48 & -73:40:07 & 305.69 & -43.31 & 49.8 & 0.058 & 3.56 \\
J002335-735529 & 30665 & 00:23:35 & -73:55:29 & 305.57 & -43.06 & 61.6 & 0.019 & 3.57 \\
J002337-735529 & 30665 & 00:23:37 & -73:55:30 & 305.56 & -43.06 & 128.3 & 0.017 & 3.57 \\
J002907-735349 & 30665 & 00:29:07 & -73:53:49 & 305.05 & -43.14 & 74.9 & 0.028 & 3.57 \\
J003412-733314 & 30665 & 00:34:12 & -73:33:15 & 304.61 & -43.52 & 49.8 & 0.030 & 3.59 \\
J003414-733327 & 30665 & 00:34:14 & -73:33:28 & 304.61 & -43.51 & 292.0 & 0.020 & 3.59 \\
J003424-721143 & 30665 & 00:34:25 & -72:11:43 & 304.77 & -44.87 & 156.8 & 0.019 & 3.55 \\
J003749-735127 & 30665 & 00:37:50 & -73:51:27 & 304.23 & -43.24 & 20.0 & 0.104 & 3.59 \\
J003824-742211 

# Full version of Table 1 in the paper for 462 lines of sight

The full version of Table 1 in the paper for 462 lines of sight.

A .CSV file is also included (result_table_dr3_integrated_properties.csv)

In [115]:
# Table 1 (FULL) in the paper for 462 lines of sight
for i, (srci, fieldi, rai, deci, li, bi, fluxi, noisei, tskyi) in enumerate(zip(srcnames, srcfields, srcra, srcdec, srcgl, srcgb, srcflux, tau_noise_per_channel, srctsky)):
    print(i, f'{srci} & {fieldi} & {rai} & {deci} & {li:0.2f} & {bi:0.2f} & {fluxi:0.1f} & {noisei:0.3f} & {tskyi:0.2f} \\\\')

0 J001424-733911 & 30665 & 00:14:24 & -73:39:11 & 306.49 & -43.22 & 179.5 & 0.015 & 3.56 \\
1 J002144-741500 & 30665 & 00:21:44 & -74:14:59 & 305.67 & -42.72 & 132.4 & 0.028 & 3.56 \\
2 J002223-742825 & 30665 & 00:22:23 & -74:28:26 & 305.56 & -42.51 & 206.7 & 0.019 & 3.56 \\
3 J002248-734007 & 30665 & 00:22:48 & -73:40:07 & 305.69 & -43.31 & 49.8 & 0.058 & 3.56 \\
4 J002335-735529 & 30665 & 00:23:35 & -73:55:29 & 305.57 & -43.06 & 61.6 & 0.019 & 3.57 \\
5 J002337-735529 & 30665 & 00:23:37 & -73:55:30 & 305.56 & -43.06 & 128.3 & 0.017 & 3.57 \\
6 J002907-735349 & 30665 & 00:29:07 & -73:53:49 & 305.05 & -43.14 & 74.9 & 0.028 & 3.57 \\
7 J003412-733314 & 30665 & 00:34:12 & -73:33:15 & 304.61 & -43.52 & 49.8 & 0.030 & 3.59 \\
8 J003414-733327 & 30665 & 00:34:14 & -73:33:28 & 304.61 & -43.51 & 292.0 & 0.020 & 3.59 \\
9 J003424-721143 & 30665 & 00:34:25 & -72:11:43 & 304.77 & -44.87 & 156.8 & 0.019 & 3.55 \\
10 J003749-735127 & 30665 & 00:37:50 & -73:51:27 & 304.23 & -43.24 & 20.0 & 0.104 & 

In [116]:
shuffled_df['tau_noise_per_channel'] = tau_noise_per_channel
print('nrow, ncol:', shuffled_df.shape)
shuffled_df.head()

nrow, ncol: (462, 45)


,comp_name,sbid,uniq_name,ra,dec,glon,glat,in_bridge,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,semi_maj_axis,semi_min_axis,pa,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,n_h,ew,sigew,ewfit,sigewfit,nhicnm,signhicnm,nhiwnm,signhiwnm,fcnm,sigfcnm,Rfactor,sigRfactor,nhi,signhi,nhi_thin,signhi_thin,ncnm,tsky,flux_int,ra_hr,tau_noise_per_channel
413,J001424-733911,30665,J001424-733911-30665,3.601159,-73.653109,306.494335,-43.218047,0,A,184.315994,0.001285,0.014211,0.223862,3.712036,1.047509,91.0,5.545,4.165,163.320007,0.823646,-1.913442,0.194014,0.020262,2.490387e+20,0.878501,0.551155,0.551155,0.053517,0.969827,0.137638,1.760664,0.264103,0.355924,0.063481,1.032038,0.123268,2.730491,0.297817,2.646857,0.110566,2,3.555342,179.488998,0.240077,0.014621
415,J002144-741500,30665,J002144-741500-30665,5.433208,-74.249829,305.669096,-42.720187,0,B,122.134003,0.000539,0.027375,0.260426,2.143209,1.082854,56.0,5.250,4.900,162.210007,0.822428,123.920952,0.195495,0.038545,1.942457e+21,1.021457,0.633919,0.633919,0.055079,0.815584,0.127427,2.348561,0.379008,0.256970,0.051081,1.003609,0.139190,3.164146,0.399856,3.152547,0.179060,2,3.561425,132.397995,0.362214,0.028466
269,J002223-742825,30665,J002223-742825-30665,5.597676,-74.473760,305.562542,-42.505801,0,B,103.077003,0.000306,0.018357,0.183846,1.969772,1.061906,74.0,7.440,6.395,123.370003,0.878060,-2.224990,0.130040,0.023986,1.508365e+21,0.812375,0.526386,0.526386,0.054060,0.864712,0.083073,2.044641,0.235230,0.300266,0.038231,1.029805,0.093871,2.909353,0.249468,2.823752,0.085782,2,3.561425,206.740997,0.373178,0.019057
258,J002248-734007,30665,J002248-734007-30665,5.700254,-73.668709,305.692177,-43.305728,0,B,49.626999,0.000237,0.055189,0.442006,1.120147,1.208479,87.0,5.000,4.765,13.430000,0.766473,-1.893242,0.265956,0.081483,1.130605e+21,1.319089,0.657144,0.657144,0.049505,1.070524,0.115079,2.084937,0.180244,0.342519,0.043122,1.059104,0.084394,3.155461,0.213848,2.971679,0.125477,1,3.564467,49.803001,0.380017,0.057622
154,J002335-735529,30665,J002335-735529-30665,5.895892,-73.924770,305.565463,-43.060758,0,B,62.061001,0.000693,0.018277,0.186595,1.639281,1.070699,126.0,5.120,4.595,168.520004,0.884104,-1.019440,0.123180,0.024585,1.914016e+21,0.273286,0.319862,0.319862,0.028930,0.419789,0.076517,2.215657,0.221813,0.160020,0.032376,0.933793,0.093914,2.635446,0.234640,2.824470,0.128897,1,3.567508,61.556000,0.393059,0.018767


In [6]:
csv_filename = hires_results / 'result_table_dr3_integrated_properties.csv' # DR3
if False:
    shuffled_df.to_csv(csv_filename, index=False)

In [7]:
# All sightlines
det_df = pd.read_csv(csv_filename)

nrows, ncols = det_df.shape
print('Number of rows: %i, columns %i. ' %(nrows, ncols))
det_df.head(5)

Number of rows: 462, columns 45. 


,comp_name,sbid,uniq_name,ra,dec,glon,glat,in_bridge,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,semi_maj_axis,semi_min_axis,pa,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,n_h,ew,sigew,ewfit,sigewfit,nhicnm,signhicnm,nhiwnm,signhiwnm,fcnm,sigfcnm,Rfactor,sigRfactor,nhi,signhi,nhi_thin,signhi_thin,ncnm,tsky,flux_int,ra_hr,tau_noise_per_channel
0,J001424-733911,30665,J001424-733911-30665,3.601159,-73.653109,306.494335,-43.218047,0,A,184.315994,0.001285,0.014211,0.223862,3.712036,1.047509,91.0,5.545,4.165,163.320007,0.823646,-1.913442,0.194014,0.020262,2.490387e+20,0.878501,0.551155,0.551155,0.053517,0.969827,0.137638,1.760664,0.264103,0.355924,0.063481,1.032038,0.123268,2.730491,0.297817,2.646857,0.110566,2,3.555342,179.488998,0.240077,0.014621
1,J002144-741500,30665,J002144-741500-30665,5.433208,-74.249829,305.669096,-42.720187,0,B,122.134003,0.000539,0.027375,0.260426,2.143209,1.082854,56.0,5.250,4.900,162.210007,0.822428,123.920952,0.195495,0.038545,1.942457e+21,1.021457,0.633919,0.633919,0.055079,0.815584,0.127427,2.348561,0.379008,0.256970,0.051081,1.003609,0.139190,3.164146,0.399856,3.152547,0.179060,2,3.561425,132.397995,0.362214,0.028466
2,J002223-742825,30665,J002223-742825-30665,5.597676,-74.473760,305.562542,-42.505801,0,B,103.077003,0.000306,0.018357,0.183846,1.969772,1.061906,74.0,7.440,6.395,123.370003,0.878060,-2.224990,0.130040,0.023986,1.508365e+21,0.812375,0.526386,0.526386,0.054060,0.864712,0.083073,2.044641,0.235230,0.300266,0.038231,1.029805,0.093871,2.909353,0.249468,2.823752,0.085782,2,3.561425,206.740997,0.373178,0.019057
3,J002248-734007,30665,J002248-734007-30665,5.700254,-73.668709,305.692177,-43.305728,0,B,49.626999,0.000237,0.055189,0.442006,1.120147,1.208479,87.0,5.000,4.765,13.430000,0.766473,-1.893242,0.265956,0.081483,1.130605e+21,1.319089,0.657144,0.657144,0.049505,1.070524,0.115079,2.084937,0.180244,0.342519,0.043122,1.059104,0.084394,3.155461,0.213848,2.971679,0.125477,1,3.564467,49.803001,0.380017,0.057622
4,J002335-735529,30665,J002335-735529-30665,5.895892,-73.924770,305.565463,-43.060758,0,B,62.061001,0.000693,0.018277,0.186595,1.639281,1.070699,126.0,5.120,4.595,168.520004,0.884104,-1.019440,0.123180,0.024585,1.914016e+21,0.273286,0.319862,0.319862,0.028930,0.419789,0.076517,2.215657,0.221813,0.160020,0.032376,0.933793,0.093914,2.635446,0.234640,2.824470,0.128897,1,3.567508,61.556000,0.393059,0.018767
